In [1]:
import pandas as pd
import os

from utils import *

In [2]:
start_date = "2025-08-18"
end_date = "2025-08-30"
l_exchanges = ["shfe", "ine"]
server = "moneymaking"
l_trading_days = get_trading_days(start_date, end_date)

In [3]:
l_hist = [d for d in l_trading_days if d <= SPLIT_DATE]
l_prod = [d for d in l_trading_days if d > SPLIT_DATE]

In [4]:
import numpy as np
if len(l_prod)> 0:
    
    with open(METADATA_PATH, "r", encoding="utf-8") as f:
        metadata = json.load(f)
    
    for exchange in l_exchanges:
        for date in l_prod:
            gz_file_path = os.path.join(PROD_MD_CSV_ROOT, exchange, f"{exchange}_{date.replace('-', '')}.csv.gz")
            df_day = load_csv_from_gz(gz_file_path)
            
            df_day = df_day[df_day.InstrumentID.str.match(FUTURE_NAME_PATTERN_MAP[exchange])]
            df_day = df_day[df_day.UpdateTime.map(in_trading_hour)]
            df_day["InstrumentRoot"] = df_day["InstrumentID"].str.extract(r"^([a-zA-Z]+)")

            d_vr = get_volume_rank_dict(df_day[["InstrumentRoot", "InstrumentID", "Volume"]])
            trading_hour_date_map = get_trading_hour_date_map(date)
            
            with open(os.path.join(VOLUME_RANK_DIR_ROOT, exchange, f"{date}.json"), "w", encoding="utf-8") as f:
                json.dump(d_vr, f, ensure_ascii=False, indent=4)
    
            for ins_root in d_vr.keys():
                price_step = metadata[ins_root]["price_step"]
                lot_size = metadata[ins_root]["lot_size"]
                for ins_id in d_vr[ins_root].keys():
                    volume_rank = d_vr[ins_root][ins_id]["rank"]
                    df = df_day[df_day.InstrumentID == ins_id].copy()
                    
                    df["server_timestamp"] = (df.RecvSec * 1000000000 + df.RecvNanosec).astype(np.int64)
                    df["instrument"] = df.InstrumentID.astype(str)
                    
                    df["UpdateMillisecStr"] = df["UpdateMillisec"].astype(str).str.zfill(3)
                    df["UpdateTimeFull"] = df["UpdateTime"] + "." + df["UpdateMillisecStr"]
                    df["exchange_timestamp_str"] = df["exchange_timestamp_str"] = df["UpdateTimeFull"].map(
                        lambda x: build_exchange_ts_str(trading_hour_date_map, x))
                    df["exchange_timestamp"] = (pd.to_datetime(df["exchange_timestamp_str"], format="%Y-%m-%d %H:%M:%S.%f")).astype(np.int64)
                    
                    df["acc_volume"] = df.Volume.fillna(-1).astype(int)
                    df["acc_turnover"] = (df.Turnover/price_step/lot_size).fillna(-1).astype(np.int64)
                    df["open_interest"] = df.OpenInterest.fillna(-1).astype(int)
                    df["last_price"] = (df.LastPrice/price_step).fillna(-1).astype(int)
                    df["last_volume"] = 1 if exchange != "dce" else df.LastVolume.fillna(-1)
                    df["bid0_price"] = (df.BidPrice1/price_step).fillna(-1).astype(int)
                    df["bid1_price"] = (df.BidPrice2/price_step).fillna(-1).astype(int)
                    df["bid2_price"] = (df.BidPrice3/price_step).fillna(-1).astype(int)
                    df["bid3_price"] = (df.BidPrice4/price_step).fillna(-1).astype(int)                    
                    df["bid4_price"] = (df.BidPrice5/price_step).fillna(-1).astype(int)
                    df["ask0_price"] = (df.AskPrice1/price_step).fillna(-1).astype(int)
                    df["ask1_price"] = (df.AskPrice2/price_step).fillna(-1).astype(int)
                    df["ask2_price"] = (df.AskPrice3/price_step).fillna(-1).astype(int)
                    df["ask3_price"] = (df.AskPrice4/price_step).fillna(-1).astype(int)                    
                    df["ask4_price"] = (df.AskPrice5/price_step).fillna(-1).astype(int)    
                    df["bid0_qty"] = df.BidVolume1.fillna(-1).astype(int)
                    df["bid1_qty"] = df.BidVolume2.fillna(-1).astype(int)
                    df["bid2_qty"] = df.BidVolume3.fillna(-1).astype(int)
                    df["bid3_qty"] = df.BidVolume4.fillna(-1).astype(int)                    
                    df["bid4_qty"] = df.BidVolume5.fillna(-1).astype(int)
                    df["ask0_qty"] = df.AskVolume1.fillna(-1).astype(int)
                    df["ask1_qty"] = df.AskVolume2.fillna(-1).astype(int)
                    df["ask2_qty"] = df.AskVolume3.fillna(-1).astype(int)
                    df["ask3_qty"] = df.AskVolume4.fillna(-1).astype(int)                    
                    df["ask4_qty"] = df.AskVolume5.fillna(-1).astype(int)                       
                    if 'UpperLimitPrice' in df.columns:
                        df["upper_limit_price"] = (df.UpperLimitPrice/price_step).fillna(-1).astype(int)
                        df["lower_limit_price"] = (df.LowerLimitPrice/price_step).fillna(-1).astype(int)
                        df["open_price"] = (df.OpenPrice/price_step).fillna(-1).astype(int)
                        df["close_price"] = (df.ClosePrice/price_step).fillna(-1).astype(int)
                        df["highest_price"] = (df.HighestPrice/price_step).fillna(-1).astype(int)
                        df["lowest_price"] = (df.LowestPrice/price_step).fillna(-1).astype(int)
                    else:
                        df["upper_limit_price"] = -1
                        df["lower_limit_price"] = -1
                        df["open_price"] = -1
                        df["close_price"] = -1
                        df["highest_price"] = -1
                        df["lowest_price"] = -1
                    
                    df_output = df[RES_MD_COLUMNS].copy()
                    df_output.sort_values(by="exchange_timestamp", inplace=True)
                    df_output.reset_index(drop=True, inplace=True)
                    output_dir = os.path.join(RES_MD_ROOT, exchange, f"{ins_root}{str(volume_rank)}")
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    output_name = os.path.join(output_dir, f"{date}.parquet")
                    df_output.to_parquet(output_name)
               